In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
# 드라이버 실행
driver = webdriver.Chrome()

In [3]:
# 기업 리스트 가져오기 & reshaping

jobplanet_excel = pd.read_excel("잡플래닛_기업리스트.xlsx", index_col = 0)

jobplanet_reshape = pd.melt(jobplanet_excel)
jobplanet_reshape.head()

,variable,value
0,서비스업,스타벅스커피코리아(주)
1,서비스업,한국맥도날드(유)
2,서비스업,롯데리아 (매장)
3,서비스업,(주)하나투어
4,서비스업,(주)유베이스


In [4]:
# 캐치 URL 가져오기 & 수정

# part1
data = pd.read_csv("urls_null.csv", sep=",")
part1 = pd.Series(list(data))
part1[198] = 'https://www.catch.co.kr/Comp/CompSummary/IW7328'
part1[443] = '방송통신대학교'
# part1

# part2
data = pd.read_csv("urls_2.csv", sep=",")
pd.melt(data).value
part2 = pd.melt(data)
# part2.value[443:]
# jobplanet_reshape[443:]
# part2.value

0      https://www.catch.co.kr/Comp/CompSummary/827461
1      https://www.catch.co.kr/Comp/CompSummary/310441
2      https://www.catch.co.kr/Comp/CompSummary/620475
3      https://www.catch.co.kr/Comp/CompSummary/021987
4      https://www.catch.co.kr/Comp/CompSummary/111010
                            ...                       
995    https://www.catch.co.kr/Comp/CompSummary/272015
996    https://www.catch.co.kr/Comp/CompSummary/001545
997    https://www.catch.co.kr/Comp/CompSummary/900436
998                                         ������ȭȸ��
999    https://www.catch.co.kr/Comp/CompSummary/033080
Length: 1000, dtype: object

In [5]:
# 재무정보 페이지로 이동

index = 0
for i in total:
    if i[:4] == 'http':
        total[index] = i.replace('CompSummary', 'CompInfo')
    index += 1    
total

0      https://www.catch.co.kr/Comp/CompInfo/827461
1      https://www.catch.co.kr/Comp/CompInfo/310441
2      https://www.catch.co.kr/Comp/CompInfo/620475
3      https://www.catch.co.kr/Comp/CompInfo/021987
4      https://www.catch.co.kr/Comp/CompInfo/111010
                           ...                     
995    https://www.catch.co.kr/Comp/CompInfo/272015
996    https://www.catch.co.kr/Comp/CompInfo/001545
997    https://www.catch.co.kr/Comp/CompInfo/900436
998                                      ������ȭȸ��
999    https://www.catch.co.kr/Comp/CompInfo/033080
Length: 1000, dtype: object

In [23]:
# 재무안정성 정보 수집
name = []
a = []
b = []
c = []

index = 23

for i in total[24:]:
    if i[0:4] != 'http':
        index += 1
        name.append(jobplanet_reshape.value[index])
        a.append(None)
        b.append(None)
        c.append(None)
    else:
        index += 1
        try:
            driver.get(i)
            time.sleep(3)
        except:
            name.append(jobplanet_reshape.value[index])
            a.append(None)
            b.append(None)
            c.append(None)
        try:
            finance = driver.find_elements_by_class_name('txt')[1].text.split('\n')
            if len(finance) != 3:
                name.append(jobplanet_reshape.value[index])
                a.append(None)
                b.append(None)
                c.append(None)     
            else:
                name.append(jobplanet_reshape.value[index])
                a.append(finance[0])
                b.append(finance[1])
                c.append(finance[2])
        except:
            name.append(jobplanet_reshape.value[index])
            a.append(None)
            b.append(None)
            c.append(None)

total_data = pd.DataFrame()

total_data['회사명'] = pd.Series(name)
total_data['부채비율'] = pd.Series(a)
total_data['당좌비율'] = pd.Series(b)
total_data['신용등급'] = pd.Series(c)

total_data.to_excel("재무안정성_캐치.xlsx", index=True)

In [24]:
total_data

,회사명,부채비율,당좌비율,신용등급
0,한국마사회,None,None,None
1,케이에이(주),부채비율: 33.5%,당좌비율: 41.0%,신용등급: 보통
2,현대그린푸드(주),부채비율: 25.9%,당좌비율: 196.4%,신용등급: 최상
3,에이디티캡스(주),None,None,None
4,케이텍맨파워(주),부채비율: 126.8%,당좌비율: 147.2%,신용등급: 양호
...,...,...,...,...
971,서울산업진흥원,서울 중소기업 성장지원사업 영위,,"대표 사업 : 스타트업 지원 프로그램, 산업 활성화 프로그램"
972,한국식품연구원,신소재·신공정 연구 개발,전통식품의 세계화 연구개발,"식품 분석, 정보, 표준화 및 기반조성 연구개발"
973,법무법인세종,부채비율: 120.1%,당좌비율: 374.3%,신용등급: 보통
974,세종문화회관,None,None,None
